In [2]:
import re

from bs4 import BeautifulSoup 
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary, MmCorpus
from langdetect import detect
import numpy as np
import pandas as pd
try:
    import spacy
except: 
    !pip install spacy #--user
finally:
    import spacy
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#import bokeh.plotting as bp
#from bokeh.plotting import save
#from bokeh.models import HoverTool

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#!pip install pyLDAvis --user 
import pyLDAvis.sklearn
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore")

## Final LDA 

In [3]:
meetup_topcats_samp = pd.read_csv('processed_df_no_badwords.csv')

In [4]:
meetup_topcats_samp.head()

,Unnamed: 0,Unnamed: 0.1,description,category_bin,lang_ok,processed_description
0,0,0,<p>FOOD AND FELLOWSHIP | 5 PM</p> \n<p>CONCERT...,27,english,food fellowship concert may us morn star churc...
1,1,1,<p><b>【WhyNot!?JAPAN + MeetUp Collaboration Ev...,28,english,whynot japan collabor peopl come whynot japan ...
2,2,2,<p>TENTATIVE<br></p>\n<p><br></p>\n<p>10.30 - ...,31,english,tentat introduct orient session break network ...
3,3,3,"<p><span>Hey explorersssss, what's up?</span><...",15,english,hey explorersssss say night view hong kong vic...
4,4,4,"<p>Free, unauditioned, collaborative pop choir...",18,english,unaudit collabor pop choir identifi women see ...


In [5]:
meetup_topcats_samp=meetup_topcats_samp[meetup_topcats_samp.processed_description.isna()!=True]

In [6]:
tf_vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english', max_features = 1000)

In [7]:
vectorized = tf_vectorizer.fit_transform(meetup_topcats_samp.processed_description)
vectorized.shape

vectorized.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
lda_model = LatentDirichletAllocation(n_components=33, random_state=0, n_jobs=4)

In [9]:
X_topics = lda_model.fit_transform(vectorized)

In [10]:
word_dict = {}
for topic_idx, topic in enumerate(lda_model.components_):
    word_dict[topic_idx] = [tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[:-10 - 1:-1]]

pd.DataFrame(word_dict).T

,0,1,2,3,4,5,6,7,8,9
0,class,danc,beginn,learn,salsa,basic,student,cours,week,lesson
1,walk,station,run,train,rout,climb,mile,meet,park,hill
2,trip,day,tour,paddl,travel,kayak,boat,lake,camp,adventur
3,danc,music,art,song,sing,draw,artist,class,lesson,fun
4,danc,parti,drink,night,bar,music,band,friend,fun,beer
5,movi,film,festiv,museum,star,watch,time,meet,seat,art
6,coffe,tea,chat,race,meet,cafe,leagu,concert,let,shop
7,let,meet,vote,know,hope,hi,look,time,everyon,make
8,hike,trail,park,mile,walk,water,bring,road,creek,meet
9,meet,month,mask,soon,project,everi,tuesday,wednesday,communiti,volunt


In [11]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='pcoa')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
31    -0.107937  0.019654       1        1  5.339777
28    -0.119913  0.023268       2        1  5.104361
4      0.050696  0.026303       3        1  4.868652
8      0.108249 -0.053224       4        1  4.816202
17    -0.086294 -0.029560       5        1  4.784439
2      0.097947  0.028526       6        1  4.742850
22    -0.118977  0.056414       7        1  4.243442
27    -0.098854  0.003723       8        1  3.939925
29     0.022118 -0.033269       9        1  3.553890
24    -0.086630  0.037879      10        1  3.393352
19     0.021854  0.034597      11        1  3.311709
23     0.043480 -0.082463      12        1  3.078218
0     -0.038779 -0.055463      13        1  3.075572
5      0.002970  0.043723      14        1  2.963760
32    -0.048452  0.012522      15        1  2.776837
12    -0.007539 -0.094834      16        1  2.737495
9     -0.024392 -0.002001      17        1  2.699853
14    -0.051329 -0.014837      18        1  2.649808
21    -0.079279  0.033200      19        1  2.643797
15     0.078413 -0.004138      20        1  2.632692
7      0.056595 -0.005831      21        1  2.585823
3     -0.031403 -0.003308      22        1  2.515254
26     0.104299 -0.074653      23        1  2.444004
16    -0.073020  0.050684      24        1  2.439399
20     0.009414  0.044873      25        1  2.401323
13     0.103202 -0.015932      26        1  2.360121
1      0.129243 -0.063840      27        1  2.344834
18    -0.015150 -0.005428      28        1  2.314087
25    -0.150503  0.118092      29        1  1.834590
6      0.030300 -0.010855      30        1  1.751514
30     0.296338  0.236529      31        1  1.483867
11    -0.063202  0.026915      32        1  1.446930
10     0.046537 -0.247267      33        1  0.721625, topic_info=    Category         Freq        Term        Total  loglift  logprob
777  Default  2063.000000        sail  2063.000000  30.0000  30.0000
761  Default  2283.000000        ride  2283.000000  29.0000  29.0000
161  Default  4574.000000       class  4574.000000  28.0000  28.0000
903  Default  1410.000000        test  1410.000000  27.0000  27.0000
382  Default  3072.000000        game  3072.000000  26.0000  26.0000
235  Default  2544.000000        danc  2544.000000  25.0000  25.0000
774  Default  2119.000000         run  2119.000000  24.0000  24.0000
427  Default  2100.000000        hike  2100.000000  23.0000  23.0000
779  Default  1065.000000      sailor  1065.000000  22.0000  22.0000
599  Default  1641.000000      nation  1641.000000  21.0000  21.0000
247  Default  1021.000000    descript  1021.000000  20.0000  20.0000
997  Default  2047.000000        yoga  2047.000000  19.0000  19.0000
119  Default  2701.000000        busi  2701.000000  18.0000  18.0000
565  Default  2181.000000       medit  2181.000000  17.0000  17.0000
674  Default  1571.000000      player  1571.000000  16.0000  16.0000
673  Default  2898.000000        play  2898.000000  15.0000  15.0000
989  Default   960.000000     workout   960.000000  14.0000  14.0000
961  Default  2691.000000        walk  2691.000000  13.0000  13.0000
640  Default   896.000000       paint   896.000000  12.0000  12.0000
641  Default  3658.000000        park  3658.000000  11.0000  11.0000
94   Default   953.000000        bike   953.000000  10.0000  10.0000
922  Default  1499.000000       trail  1499.000000   9.0000   9.0000
607  Default  2029.000000     network  2029.000000   8.0000   8.0000
575  Default  1533.000000        mile  1533.000000   7.0000   7.0000
415  Default  1253.000000        heal  1253.000000   6.0000   6.0000
220  Default  1025.000000       court  1025.000000   5.0000   5.0000
855  Default   821.000000     startup   821.000000   4.0000   4.0000
595  Default   877.000000        movi   877.000000   3.0000   3.0000
236  Default  1046.000000        data  1046.000000   2.0000   2.0000
175  Default  1203.000000       

In [12]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
31     0.019552 -0.147637       1        1  5.339777
28     0.126647 -0.136577       2        1  5.104361
4      0.057112  0.090705       3        1  4.868652
8     -0.052355  0.181193       4        1  4.816202
17    -0.092952 -0.178913       5        1  4.784439
2     -0.103926  0.068825       6        1  4.742850
22     0.140295 -0.098685       7        1  4.243442
27    -0.036421 -0.177635       8        1  3.939925
29    -0.031347  0.046071       9        1  3.553890
24     0.121682 -0.057554      10        1  3.393352
19    -0.040755  0.007497      11        1  3.311709
23    -0.173375  0.040465      12        1  3.078218
0     -0.115870 -0.096446      13        1  3.075572
5      0.050164  0.021313      14        1  2.963760
32     0.039930 -0.039599      15        1  2.776837
12    -0.158466 -0.080059      16        1  2.737495
9      0.108174  0.023110      17        1  2.699853
14     0.022176 -0.092083      18        1  2.649808
21     0.073965 -0.129884      19        1  2.643797
15    -0.071047  0.105920      20        1  2.632692
7      0.087703  0.150683      21        1  2.585823
3     -0.064288 -0.081841      22        1  2.515254
26    -0.119468  0.172503      23        1  2.444004
16     0.152167 -0.034251      24        1  2.439399
20     0.127689  0.071222      25        1  2.401323
13    -0.027398  0.124139      26        1  2.360121
1     -0.070507  0.180977      27        1  2.344834
18    -0.046567 -0.037908      28        1  2.314087
25     0.275797 -0.105643      29        1  1.834590
6      0.028220  0.069351      30        1  1.751514
30    -0.445623  0.042306      31        1  1.483867
11     0.027252 -0.241103      32        1  1.446930
10     0.191839  0.339538      33        1  0.721625, topic_info=    Category         Freq        Term        Total  loglift  logprob
777  Default  2063.000000        sail  2063.000000  30.0000  30.0000
761  Default  2283.000000        ride  2283.000000  29.0000  29.0000
161  Default  4574.000000       class  4574.000000  28.0000  28.0000
903  Default  1410.000000        test  1410.000000  27.0000  27.0000
382  Default  3072.000000        game  3072.000000  26.0000  26.0000
235  Default  2544.000000        danc  2544.000000  25.0000  25.0000
774  Default  2119.000000         run  2119.000000  24.0000  24.0000
427  Default  2100.000000        hike  2100.000000  23.0000  23.0000
779  Default  1065.000000      sailor  1065.000000  22.0000  22.0000
599  Default  1641.000000      nation  1641.000000  21.0000  21.0000
247  Default  1021.000000    descript  1021.000000  20.0000  20.0000
997  Default  2047.000000        yoga  2047.000000  19.0000  19.0000
119  Default  2701.000000        busi  2701.000000  18.0000  18.0000
565  Default  2181.000000       medit  2181.000000  17.0000  17.0000
674  Default  1571.000000      player  1571.000000  16.0000  16.0000
673  Default  2898.000000        play  2898.000000  15.0000  15.0000
989  Default   960.000000     workout   960.000000  14.0000  14.0000
961  Default  2691.000000        walk  2691.000000  13.0000  13.0000
640  Default   896.000000       paint   896.000000  12.0000  12.0000
641  Default  3658.000000        park  3658.000000  11.0000  11.0000
94   Default   953.000000        bike   953.000000  10.0000  10.0000
922  Default  1499.000000       trail  1499.000000   9.0000   9.0000
607  Default  2029.000000     network  2029.000000   8.0000   8.0000
575  Default  1533.000000        mile  1533.000000   7.0000   7.0000
415  Default  1253.000000        heal  1253.000000   6.0000   6.0000
220  Default  1025.000000       court  1025.000000   5.0000   5.0000
855  Default   821.000000     startup   821.000000   4.0000   4.0000
595  Default   877.000000        movi   877.000000   3.0000   3.0000
236  Default  1046.000000        data  1046.000000   2.0000   2.0000
175  Default  1203.000000       

In [13]:
pyLDAvis.sklearn.prepare(lda_model, vectorized, tf_vectorizer, mds='tsne') #remember: topic 25 is shit

PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
31    -25.428339 -75.677811       1        1  5.339777
28    -24.860235 -46.333241       2        1  5.104361
4      10.174921  27.025423       3        1  4.868652
8      67.639793 -31.273979       4        1  4.816202
17    -78.544724 -30.275671       5        1  4.784439
2      46.903709 -12.575776       6        1  4.742850
22      0.583713 -41.885429       7        1  4.243442
27    -56.744312 -59.332962       8        1  3.939925
29    -20.012163  62.374077       9        1  3.553890
24     40.453369  65.720131      10        1  3.393352
19      8.828137  52.831997      11        1  3.311709
23    -48.599361  63.496048      12        1  3.078218
0     -59.153290  -0.201340      13        1  3.075572
5      49.724926  15.467679      14        1  2.963760
32     34.654079  37.150494      15        1  2.776837
12    -84.982330  12.893377      16        1  2.737495
9     -18.430494  34.928726      17        1  2.699853
14    -21.227749 -16.088711      18        1  2.649808
21      6.542944 -69.685707      19        1  2.643797
15     27.077711 -29.279526      20        1  2.632692
7       4.977262  83.674568      21        1  2.585823
3       5.013349 -13.164265      22        1  2.515254
26     43.137550 -53.218189      23        1  2.444004
16    -45.677551 -26.696793      24        1  2.439399
20     24.428637   5.336960      25        1  2.401323
13    -31.230642   9.854044      26        1  2.360121
1      78.532936  -2.125448      27        1  2.344834
18    -50.268314  31.028822      28        1  2.314087
25     78.119995 -65.531883      29        1  1.834590
6      -6.389129   9.869995      30        1  1.751514
30     39.379883 -94.879623      31        1  1.483867
11     72.395599  41.363972      32        1  1.446930
10    -87.357834  54.656338      33        1  0.721625, topic_info=    Category         Freq        Term        Total  loglift  logprob
777  Default  2063.000000        sail  2063.000000  30.0000  30.0000
761  Default  2283.000000        ride  2283.000000  29.0000  29.0000
161  Default  4574.000000       class  4574.000000  28.0000  28.0000
903  Default  1410.000000        test  1410.000000  27.0000  27.0000
382  Default  3072.000000        game  3072.000000  26.0000  26.0000
235  Default  2544.000000        danc  2544.000000  25.0000  25.0000
774  Default  2119.000000         run  2119.000000  24.0000  24.0000
427  Default  2100.000000        hike  2100.000000  23.0000  23.0000
779  Default  1065.000000      sailor  1065.000000  22.0000  22.0000
599  Default  1641.000000      nation  1641.000000  21.0000  21.0000
247  Default  1021.000000    descript  1021.000000  20.0000  20.0000
997  Default  2047.000000        yoga  2047.000000  19.0000  19.0000
119  Default  2701.000000        busi  2701.000000  18.0000  18.0000
565  Default  2181.000000       medit  2181.000000  17.0000  17.0000
674  Default  1571.000000      player  1571.000000  16.0000  16.0000
673  Default  2898.000000        play  2898.000000  15.0000  15.0000
989  Default   960.000000     workout   960.000000  14.0000  14.0000
961  Default  2691.000000        walk  2691.000000  13.0000  13.0000
640  Default   896.000000       paint   896.000000  12.0000  12.0000
641  Default  3658.000000        park  3658.000000  11.0000  11.0000
94   Default   953.000000        bike   953.000000  10.0000  10.0000
922  Default  1499.000000       trail  1499.000000   9.0000   9.0000
607  Default  2029.000000     network  2029.000000   8.0000   8.0000
575  Default  1533.000000        mile  1533.000000   7.0000   7.0000
415  Default  1253.000000        heal  1253.000000   6.0000   6.0000
220  Default  1025.000000       court  1025.000000   5.0000   5.0000
855  Default   821.000000     startup   821.000000   4.0000   4.0000
595  Default   877.000000        movi   877.000000   3.0000   3.0000
236  Default  1046.000000      